# Assignment 2 - Fama and MacBeth (Jaakko Wallenius, Quantitative Finance 28E35600)

**Note**: Pandas may fail when using Windows os due to memory allocation limitation (raises **MemoryError**). In case this happens, rerun the code. More on this:

https://stackoverflow.com/questions/17557074/memory-error-when-using-pandas-read-csv

In [1]:
%%capture
%run data.ipynb # The datasets are imported and prepared in the data notebook
%run testing_capm.ipynb # Portfolios from the previous assignment (computation runs for ~6min)

### From previous assignment

We use the ME portfolios and their estimated CAPM betas from previous assignment.

In [2]:
portfolios_returns.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
date,,,,,,,,,,,,,,,,,,,,
201501,-0.007181,-0.056454,-0.061759,-0.061828,-0.054846,-0.050974,-0.036410,-0.041592,-0.028135,-0.021903,-0.031285,-0.025724,-0.014178,-0.017288,-0.026940,-0.012640,-0.015240,-0.019894,-0.024454,-0.029497
201502,0.054120,0.066331,0.057818,0.101351,0.075195,0.085693,0.065800,0.076293,0.077237,0.087842,0.082795,0.076377,0.083585,0.079381,0.073652,0.070084,0.067650,0.064655,0.061519,0.060499
201503,-0.011916,0.025672,0.015851,0.013954,0.014666,0.034464,0.023158,0.010536,0.024474,0.019990,0.017127,0.010526,0.022305,0.008066,0.002323,-0.008324,-0.001534,-0.003362,-0.004184,-0.013964
201504,0.014094,0.000555,-0.006207,-0.001281,-0.006889,-0.009587,-0.008550,-0.015238,-0.021642,-0.004186,-0.013742,0.002149,0.003607,-0.001019,-0.000602,0.006019,0.000323,-0.000512,0.011122,0.010250
201505,-0.005888,0.027006,0.025659,0.010037,0.025191,0.027855,0.026427,0.041402,0.031336,0.030231,0.013905,0.022463,0.021497,0.021080,0.037947,0.018456,0.018464,-0.006684,0.027215,0.016992


In [3]:
betas

array([1.16832313, 1.16507952, 1.23600086, 1.36785012, 1.3522247 ,
       1.34587668, 1.32961017, 1.27740913, 1.33242778, 1.28055231,
       1.2593777 , 1.20540737, 1.18901791, 1.21949956, 1.39459147,
       1.13466509, 1.06297288, 0.99841957, 0.96864614, 0.97252629])

### Lagged sizes

First we calculate the average ME of stocks in each portfolio (at each month)

In [4]:
def portfolios_me_summary_month(data,date,n):
    # Helper for portfolios_me_summary
    name = 'PORT'
    d = {'date':date}
    for i in range(1,n+1):
        d[i] = data.loc[(data['date'] == date) & (data[name] == i)]['ME'].values.mean()
    return d

def portfolios_me_summary(data,n):
    # Returns DataFrame. For each date the average market equity in the quantile portfolios.
    dates = data['date'].unique()
    df = pd.DataFrame(threadify(lambda d: portfolios_me_summary_month(data,d,n),dates)).set_index('date')
    return df.sort_index(axis=1)

In [5]:
average_sizes = portfolios_me_summary(portfolios_data,20)

In [6]:
average_sizes.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
date,,,,,,,,,,,,,,,,,,,,
201501,76.782413,233.018703,366.409527,511.860385,677.740642,915.429913,1208.534112,1563.687846,1944.328260,2421.237889,2995.306309,3581.166119,4336.214771,5593.419752,7404.193199,9493.873488,13462.402654,20920.049330,34403.922296,123155.734865
201502,80.634780,249.957793,392.292203,549.882896,735.547477,1007.187679,1309.115410,1649.346237,2073.684503,2557.578097,3157.879671,3809.843612,4572.168130,5962.516478,7879.802824,10099.491284,13974.771172,22009.257142,35985.108702,128087.750354
201503,83.204201,248.767884,392.705778,565.406251,741.894729,1028.992843,1331.641817,1682.411696,2126.076508,2602.956303,3217.522406,3858.430303,4682.188301,6004.877856,7855.726255,10003.671136,13987.970045,21709.806136,35497.689185,124585.672589
201504,81.264303,251.469822,390.924812,554.017247,755.436946,1048.824852,1332.049413,1691.416661,2135.044264,2565.764884,3191.466823,3816.924449,4670.962391,6048.625391,7882.029402,10115.539562,13759.287323,21495.649469,35377.153179,126303.520696
201505,76.449939,246.379241,393.751930,549.902013,748.288868,1050.712562,1344.062969,1703.628416,2128.910662,2551.757702,3160.878894,3805.569533,4760.533965,6060.138798,7944.657071,10125.625673,13935.353823,21460.729015,36413.597442,129066.432814


Then the values are lagged by one. As a consequence we lose one observation for each portfolio (the first month 201501).

In [7]:
lagged_average_size = average_sizes.shift(1).iloc[1:]
lagged_average_size.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
date,,,,,,,,,,,,,,,,,,,,
201502,76.782413,233.018703,366.409527,511.860385,677.740642,915.429913,1208.534112,1563.687846,1944.328260,2421.237889,2995.306309,3581.166119,4336.214771,5593.419752,7404.193199,9493.873488,13462.402654,20920.049330,34403.922296,123155.734865
201503,80.634780,249.957793,392.292203,549.882896,735.547477,1007.187679,1309.115410,1649.346237,2073.684503,2557.578097,3157.879671,3809.843612,4572.168130,5962.516478,7879.802824,10099.491284,13974.771172,22009.257142,35985.108702,128087.750354
201504,83.204201,248.767884,392.705778,565.406251,741.894729,1028.992843,1331.641817,1682.411696,2126.076508,2602.956303,3217.522406,3858.430303,4682.188301,6004.877856,7855.726255,10003.671136,13987.970045,21709.806136,35497.689185,124585.672589
201505,81.264303,251.469822,390.924812,554.017247,755.436946,1048.824852,1332.049413,1691.416661,2135.044264,2565.764884,3191.466823,3816.924449,4670.962391,6048.625391,7882.029402,10115.539562,13759.287323,21495.649469,35377.153179,126303.520696
201506,76.449939,246.379241,393.751930,549.902013,748.288868,1050.712562,1344.062969,1703.628416,2128.910662,2551.757702,3160.878894,3805.569533,4760.533965,6060.138798,7944.657071,10125.625673,13935.353823,21460.729015,36413.597442,129066.432814


Next we take natural log of the lagged average MEs.

In [8]:
log_lagged_average_size = np.log(lagged_average_size)
log_lagged_average_size.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
date,,,,,,,,,,,,,,,,,,,,
201502,4.340976,5.451119,5.903752,6.238052,6.518765,6.819394,7.097163,7.354802,7.572672,7.792034,8.004802,8.183444,8.374757,8.629346,8.909802,9.158402,9.507656,9.948463,10.445926,11.721205
201503,4.389930,5.521292,5.972007,6.309705,6.600615,6.914917,7.177107,7.408134,7.637082,7.846816,8.057656,8.245343,8.427743,8.693248,8.972058,9.220240,9.545009,9.999218,10.490860,11.760471
201504,4.421298,5.516520,5.973061,6.337545,6.609207,6.936336,7.194168,7.427984,7.662034,7.864403,8.076367,8.258016,8.451521,8.700327,8.968998,9.210707,9.545953,9.985519,10.477223,11.732749
201505,4.397707,5.527323,5.968515,6.317196,6.627296,6.955426,7.194474,7.433322,7.666243,7.850012,8.068236,8.247200,8.449120,8.707586,8.972341,9.221828,9.529469,9.975606,10.473822,11.746443
201506,4.336636,5.506872,5.975721,6.309740,6.617789,6.957224,7.203452,7.440516,7.663366,7.844538,8.058605,8.244221,8.468115,8.709488,8.980255,9.222825,9.542184,9.973980,10.502698,11.768083


## Fama-MacBeth regressions

Cross-sectional regression:

$R_{p,t}=a+b_1\beta_p+b_2ln(size_{p,t-1})+\epsilon_{p,t}$

where $R_{p,t}$=portfolio $p$ return at month $t$, $\beta_p$=portfolio $p$ beta from time series regression (assignment 1) and $ln(size_{p,t-1})$=log of lagged average stock size of the
portfolio $p$.

Prepare data for the regressions

In [10]:
portfolios_returns_by_date = portfolios_returns.transpose().drop('201501',axis=1) # drop 201501 since no lagged size
log_lagged_sized_by_date = log_lagged_average_size.transpose()
dates = portfolios_returns.index.values[1:]

In [11]:
portfolios_returns_by_date.head()

date,201502,201503,201504,201505,201506,201507,201508,201509,201510,201511,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
1,0.054120,-0.011916,0.014094,-0.005888,-0.017305,-0.051594,-0.030001,-0.059364,0.016839,0.021852,...,-0.002874,0.002913,-0.089619,0.010678,-0.037615,-0.058302,0.010613,-0.015732,0.001927,0.058857
2,0.066331,0.025672,0.000555,0.027006,0.018377,-0.041802,-0.012318,-0.074372,0.047923,0.014912,...,-0.025159,0.028149,-0.065253,0.063322,-0.033034,-0.063019,0.046445,-0.026585,0.042806,0.072432
3,0.057818,0.015851,-0.006207,0.025659,0.016297,-0.050767,-0.046353,-0.059526,0.071374,0.042128,...,-0.022723,0.010574,-0.082366,0.074373,-0.011782,-0.047712,0.040932,0.010174,0.047993,0.064030
4,0.101351,0.013954,-0.001281,0.010037,0.017648,-0.031230,-0.056178,-0.087753,0.074338,0.059232,...,-0.004459,0.037348,-0.077795,0.064092,-0.001646,-0.061065,0.043958,0.025986,0.080644,0.082414
5,0.075195,0.014666,-0.006889,0.025191,0.023982,-0.039741,-0.041424,-0.048949,0.052359,0.069047,...,-0.023548,0.029463,-0.088314,0.083088,-0.005188,-0.066518,0.030667,0.019197,0.083322,0.069146


In [12]:
log_lagged_sized_by_date.head()

date,201502,201503,201504,201505,201506,201507,201508,201509,201510,201511,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
1,4.340976,4.389930,4.421298,4.397707,4.336636,4.348026,4.277650,4.241081,4.155082,4.191031,...,4.389973,4.382391,4.372660,4.164515,4.183504,4.204788,4.145309,4.202304,4.128748,4.142351
2,5.451119,5.521292,5.516520,5.527323,5.506872,5.505939,5.395811,5.343166,5.239814,5.301766,...,5.547642,5.496447,5.515444,5.353454,5.423697,5.373408,5.303952,5.369362,5.318284,5.329721
3,5.903752,5.972007,5.973061,5.968515,5.975721,5.976034,5.875077,5.822380,5.754926,5.809517,...,6.007896,5.976376,6.002625,5.851430,5.926357,5.879952,5.812000,5.872366,5.846796,5.831788
4,6.238052,6.309705,6.337545,6.317196,6.309740,6.342446,6.271168,6.222849,6.097943,6.166933,...,6.415736,6.407525,6.427178,6.272739,6.356553,6.333964,6.223965,6.312512,6.266111,6.277482
5,6.518765,6.600615,6.609207,6.627296,6.617789,6.630411,6.615276,6.546181,6.444644,6.508543,...,6.760840,6.737446,6.763353,6.619472,6.704196,6.697882,6.585190,6.665168,6.634520,6.660748


In [13]:
def cross_sectional_for_date(returns,sizes,betas,date):
    # Helper for cross_sectional_datas
    df = pd.DataFrame()
    df['R'] = returns[date]
    df['beta'] = betas
    df['loglag_size'] = sizes[date]
    return df

def cross_sectional_datas(returns,sizes,betas,dates):
    # Returns list of DataFrames (each for a date).
    # These can used in cross-sectional regressions.
    dfs = []
    for d in dates:
        dfs.append(cross_sectional_for_date(returns,sizes,betas,d))
    return dfs

In [14]:
def t_alphas_betas_tstats_portfolio(data):
    # Helper for regression_result_table
    # Returns regression results for a single cross-sectional regression
    model = ols('R ~ beta + loglag_size', data).fit()
    d = {'a' : model.params['Intercept'],
         'b_1' : model.params['beta'],
         'b_2' : model.params['loglag_size']
        }
    return d

def regression_result_table(datas):
    # Returns DataFrame of all cross-sectional regressions.
    # Assumes 'datas' is list of DataFrames (one for each cross-section)
    rows = []
    for d in datas:
        rows.append(t_alphas_betas_tstats_portfolio(d))
    return pd.DataFrame(rows)

In [15]:
dfs = cross_sectional_datas(portfolios_returns_by_date,
                            log_lagged_sized_by_date,
                            betas,
                            dates)

reg_table = regression_result_table(dfs)

## Regression results

reg_table contains all the necessary data for the Fama-MacBeth regression results.

$\hat{b}=\frac{1}{T}\sum_{t=1}^Tb_t$

$SE(\hat{b})=\frac{SE(\hat{b}_t)}{\sqrt{T}}$

In [16]:
T = reg_table['a'].size
estimates = reg_table.mean().values
errors = reg_table.std().values/np.sqrt(T)
t_stats = estimates/errors
p_vals = map(lambda stat: t.sf(abs(stat),T-1)*2,t_stats)

result_table = pd.DataFrame({'Estimate':estimates,
                             'Standard error':errors,
                             't stat':t_stats,
                             'p values':p_vals}).T
result_table.columns = ['a','b_1','b_2']

In [17]:
result_table

,a,b_1,b_2
Estimate,-0.032413,0.024502,0.002071
Standard error,0.009227,0.006859,0.000732
t stat,-3.512811,3.571928,2.830112
p values,0.000866,0.000721,0.006383


## Discussion

From the above table we see that coefficients for CAPM beta and size risk are significant. Hence we can conclude that coupling beta with size risk measure CAPM works better than without controlling size risk. However the results are not intuitive. One would expected $b_2$ to be negative.